### Read scores frome csv

In [ ]:
import os
import pandas as pd
from tqdm.auto import tqdm
from scipy.stats import pearsonr, kendalltau
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

folds = 5
data_root = '/home1/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/'
result_path = '/home1/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/results/regression_on_docking_scores.csv'
task_dict = {'I1': ('CHEMBL202', 'pIC50', '1boz', 7, 1), 'I2': ('CHEMBL3976', 'pIC50', '4ebb', 2, 4), 
             'I3': ('CHEMBL333', 'pIC50', '1ck7', 6, 2), 'I4': ('CHEMBL2971', 'pIC50', '3ugc', 3, 4), 
             'I5': ('CHEMBL279', 'pIC50', '1ywn', 3, 4), 'E1': ('CHEMBL3820', 'pEC50', '3f9m', 6, 3), 
             'E2': ('CHEMBL4422', 'pEC50', '5tzr', 3, 3), 'E3': ('CHEMBL235', 'pEC50', '1zgy', 4, 2)}
set_list = ['test', 'train_1', 'train_2', 'train_3', 'train_4', 'train_5']
# Create a dataframe to store test indexes
test_results = pd.DataFrame(columns=['Task', 'Fold', 'Val_Pearson', 'Pearson', 'RMSE', 'Kendall Tau'])
device = torch.device('cuda:0')
hidden_size = 64  # You can adjust this according to your needs
output_size = 1   # Since it's regression, output size is 1
batch_size = 32  # Adjust batch size according to your needs
# Training loop with validation and early stopping
num_epochs = 1000
patience = 30  # Number of epochs to wait for improvement before early stopping
criterion = nn.MSELoss()

# Define your regression model
class RegressionModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define function to calculate evaluation metrics
def evaluate_model(model, data_loader):
    model.eval()
    preds = []
    labels = []
    with torch.no_grad():
        for inputs, targets in data_loader:
            outputs = model(inputs.float()).squeeze()
            outputs = outputs.cpu().numpy()
            targets = targets.cpu().numpy()
            # Convert scalar output to 1D array
            if np.isscalar(outputs):
                outputs = np.array([outputs])
            if np.isscalar(targets):
                targets = np.array([targets])
            if isinstance(outputs, np.ndarray) and outputs.ndim == 0:
                outputs = np.array([outputs.item()])  # Convert single element array to scalar
            if isinstance(targets, np.ndarray) and targets.ndim == 0:
                targets = np.array([targets.item()])  # Convert single element array to scalar
            preds.extend(outputs.tolist())  # Convert to list before extending
            labels.extend(targets.tolist())  # Convert to list before extending
    preds = np.array(preds)
    labels = np.array(labels)
    pearson_corr, _ = pearsonr(preds, labels)
    rmse = np.sqrt(np.mean((preds - labels) ** 2))
    kendall_tau, _ = kendalltau(preds, labels)
    return pearson_corr, rmse, kendall_tau

for task, value in task_dict.items():
    # if task != 'I3':
    #     continue
    print(f"Task: {task}")
    target, assay, pdb, pockets, poses = value
    input_dir = f'{data_root}/{target}/{pdb}/vina/{target}_{assay}'
    input_size = pockets * poses # Define your input size based on the number of features

    for fold in range(folds):
        # if fold != 2:
        #     continue
        print(f"Fold: {fold+1}/{folds}")
        
        # Initialize your model and optimizer
        model = RegressionModel(input_size, hidden_size, output_size).to(device)
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        # Loop through input files
        train_data, train_label, val_data, val_label, test_data, test_label = [], [], [], [], [], []
        for set_name in set_list:
            data_path = input_dir + f'_{set_name}.csv'
            data_df = pd.read_csv(data_path)
            for idx, row in data_df.iterrows():
                feature, found_nan = [], False
                label = row[assay]
                # Get features (docking scores) and labels (bioactivities) for each ligand
                for pocket in range(pockets):
                    for pose in range(poses):
                        column_name = f'Pocket_{pocket+1}_Vina_Score' if task == 'I1' else f'Pocket_{pocket+1}-{pose+1}_Vina_Score'
                        score_in_column = row[column_name]
                        if np.isnan(score_in_column):
                            found_nan = True
                        else:
                            feature.append(score_in_column)
                            
                # Discard incomplete docking scores   
                if not found_nan:
                    if set_name == 'test':
                        test_data.append(torch.tensor(feature).unsqueeze(0).to(device))
                        test_label.append(torch.tensor(label).long().to(device))
                    elif set_name == f'train_{fold+1}':
                        val_data.append(torch.tensor(feature).unsqueeze(0).to(device))
                        val_label.append(torch.tensor(label).long().to(device))
                    else:
                        train_data.append(torch.tensor(feature).unsqueeze(0).to(device))
                        train_label.append(torch.tensor(label).long().to(device))
                        
        # Convert data and labels to PyTorch tensors
        train_dataset = TensorDataset(torch.stack(train_data), torch.stack(train_label))
        val_dataset = TensorDataset(torch.stack(val_data), torch.stack(val_label))
        test_dataset = TensorDataset(torch.stack(test_data), torch.stack(test_label))

        # Define DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)
        test_loader = DataLoader(test_dataset, batch_size=batch_size)

        best_valid_index = float('-inf')
        counter = 0  # Counter for early stopping
        # Training loop
        for epoch in tqdm(range(num_epochs)):
            model.train()
            running_loss = 0.0
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs.float())  # Forward pass
                loss = criterion(outputs.squeeze(), labels.float())  # Calculate the loss
                loss.backward()  # Backward pass
                optimizer.step()  # Optimize
                running_loss += loss.item() * inputs.size(0)
            epoch_loss = running_loss / len(train_loader.dataset)
            
            # Validation
            val_pearson, val_rmse, val_kendall_tau = evaluate_model(model, val_loader)
            # print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Val Pearson: {val_pearson:.4f}, Val RMSE: {val_rmse:.4f}, Val Kendall Tau: {val_kendall_tau:.4f}")
            
            # Check for early stopping
            if val_pearson > best_valid_index:
                best_valid_index = val_pearson
                counter = 0
            else:
                counter += 1
                if counter >= patience:
                    print("Early stopping...")
                    break
        
        # Evaluate on test set
        test_pearson, test_rmse, test_kendall_tau = evaluate_model(model, test_loader)
        print(f"Best Valid Pearson: {best_valid_index:.4f}, Test Pearson: {test_pearson:.4f}, Test RMSE: {test_rmse:.4f}, Test Kendall Tau: {test_kendall_tau:.4f}")

        # Append to test results dataframe
        result_dict = {
            'Task': task,
            'Fold': fold + 1,
            'Val_Pearson': best_valid_index,
            'Pearson': test_pearson,
            'RMSE': test_rmse,
            'Kendall Tau': test_kendall_tau
        }
        result_list = []
        for key, value in result_dict.items():
            result_list.append(value)
        test_results.loc[len(test_results.index)] = result_list

        # Save the dataframe to result_path at the end of each experiment
        test_results.to_csv(result_path, index=False)

Task: I1
Fold: 1/5


  0%|          | 0/1000 [00:00<?, ?it/s]

Early stopping...
Best Valid Pearson: 0.1010, Test Pearson: 0.3124, Test RMSE: 1.5286, Test Kendall Tau: 0.2482
Fold: 2/5


  0%|          | 0/1000 [00:00<?, ?it/s]

Early stopping...
Best Valid Pearson: 0.1496, Test Pearson: 0.1783, Test RMSE: 1.5076, Test Kendall Tau: 0.2685
Fold: 3/5


...

  0%|          | 0/1000 [00:00<?, ?it/s]

Early stopping...
Best Valid Pearson: 0.2839, Test Pearson: 0.1715, Test RMSE: 1.1695, Test Kendall Tau: 0.0884
Fold: 2/5


  0%|          | 0/1000 [00:00<?, ?it/s]

Early stopping...
Best Valid Pearson: 0.3751, Test Pearson: 0.1656, Test RMSE: 1.1671, Test Kendall Tau: 0.0868
Fold: 3/5


  0%|          | 0/1000 [00:00<?, ?it/s]